***GENERATED CODE FOR denialauto PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID0', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID0', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '1045', 'mean': '', 'stddev': '', 'min': '008ecc10-5900-4280-b5d6-fceb6e8d1dde', 'max': 'ff54b805-3ca5-4d22-999e-f75de7d0b71d', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID0'}, {'feature_label': 'ChargeDetailID0', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID0')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'RVUModifier', 'transformation_label': 'String Indexer'}], 'feature': 'RVUModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1045', 'mean': '26.0', 'stddev': '0.0', 'min': '26', 'max': '26', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'RVUModifier'}, {'feature_label': 'RVUModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('RVUModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1045', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1045', 'mean': '', 'stddev': '', 'min': 'AARPMedicareComplete', 'max': 'WorkersComp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerName'}, {'feature_label': 'PayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1045', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'WorkComp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerCategory'}, {'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1045', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'OtherGovernment', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerGroup'}, {'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SourceKey58', 'transformation_label': 'String Indexer'}], 'feature': 'SourceKey58', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1045', 'mean': '', 'stddev': '', 'min': 'CO109', 'max': 'PR3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SourceKey58'}, {'feature_label': 'SourceKey58', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SourceKey58')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDate', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1045', 'mean': '', 'stddev': '', 'min': '01/05/2020', 'max': '31/03/2020', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDate'}, {'feature_label': 'ReasonDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityID63', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityID63', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1045', 'mean': '', 'stddev': '', 'min': '087613d0-c7f3-4bcd-900a-abbc6ee3bdad', 'max': 'fde71f17-372e-4cde-95a7-d8b9b45a015a', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityID63'}, {'feature_label': 'FacilityID63', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityID63')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentID', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1045', 'mean': '', 'stddev': '', 'min': '0a414089-7b3a-4378-ba12-23a2d7272bcf', 'max': 'fb68b47d-bad4-42c2-9ad0-9816864ea30a', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentID'}, {'feature_label': 'DepartmentID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentID')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run denialautoHooks.ipynb
try:
	#sourcePreExecutionHook()

	whitespaceclassificationdenails = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/demo/whitespace_Classification_denails.csv', 'filename': 'whitespace_Classification_denails.csv', 'token': '', 'delimiter': ','}")
	#sourcePostExecutionHook(whitespaceclassificationdenails)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run denialautoHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(whitespaceclassificationdenails,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID0", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID0", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "", "stddev": "", "min": "008ecc10-5900-4280-b5d6-fceb6e8d1dde", "max": "ff54b805-3ca5-4d22-999e-f75de7d0b71d", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID0"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1045", "mean": "225.89", "stddev": "340.91", "min": "7.0", "max": "5154.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1045", "mean": "82.53", "stddev": "174.13", "min": "-566.3", "max": "2100.23", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1045", "mean": "1723.63", "stddev": "1192.75", "min": "1", "max": "4005", "missing": "0"}, "updatedLabel": "CPTCodeID3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiagnosisCodeID2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1045", "mean": "29936.01", "stddev": "22475.97", "min": "13501", "max": "94903", "missing": "0"}, "updatedLabel": "DiagnosisCodeID2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiagnosisCodeID1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1045", "mean": "38148.76", "stddev": "20651.01", "min": "13573", "max": "94819", "missing": "0"}, "updatedLabel": "DiagnosisCodeID1"}, {"transformationsData": [{"feature_label": "RVUModifier", "transformation_label": "String Indexer"}], "feature": "RVUModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "26.0", "stddev": "0.0", "min": "26", "max": "26", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "RVUModifier"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EnterpriseID18", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1045", "mean": "1.0", "stddev": "0.06", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "EnterpriseID18"}, {"transformationsData": [{"feature_label": "PayerName", "transformation_label": "String Indexer"}], "feature": "PayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "", "stddev": "", "min": "AARPMedicareComplete", "max": "WorkersComp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerName"}, {"transformationsData": [{"feature_label": "PayerCategory", "transformation_label": "String Indexer"}], "feature": "PayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "", "stddev": "", "min": "Aetna", "max": "WorkComp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerCategory"}, {"transformationsData": [{"feature_label": "PayerGroup", "transformation_label": "String Indexer"}], "feature": "PayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "", "stddev": "", "min": "Aetna", "max": "OtherGovernment", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerGroup"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SpecialityID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1045", "mean": "35.83", "stddev": "30.22", "min": "-1", "max": "120", "missing": "0"}, "updatedLabel": "SpecialityID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TaxonomyID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1045", "mean": "359.96", "stddev": "171.47", "min": "-1", "max": "770", "missing": "0"}, "updatedLabel": "TaxonomyID"}, {"transformationsData": [{"feature_label": "SourceKey58", "transformation_label": "String Indexer"}], "feature": "SourceKey58", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "", "stddev": "", "min": "CO109", "max": "PR3", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SourceKey58"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount59", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1045", "mean": "225.89", "stddev": "340.91", "min": "7.0", "max": "5154.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount59"}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "String Indexer"}], "feature": "ReasonDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "", "stddev": "", "min": "01/05/2020", "max": "31/03/2020", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PracticeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1045", "mean": "1.0", "stddev": "0.06", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "PracticeID"}, {"transformationsData": [{"feature_label": "FacilityID63", "transformation_label": "String Indexer"}], "feature": "FacilityID63", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "", "stddev": "", "min": "087613d0-c7f3-4bcd-900a-abbc6ee3bdad", "max": "fde71f17-372e-4cde-95a7-d8b9b45a015a", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityID63"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID66", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1045", "mean": "1723.63", "stddev": "1192.75", "min": "1", "max": "4005", "missing": "0"}, "updatedLabel": "CPTCodeID66"}, {"transformationsData": [{"feature_label": "DepartmentID", "transformation_label": "String Indexer"}], "feature": "DepartmentID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1045", "mean": "", "stddev": "", "min": "0a414089-7b3a-4378-ba12-23a2d7272bcf", "max": "fb68b47d-bad4-42c2-9ad0-9816864ea30a", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1045", "mean": "0.51", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "status"}, {"feature": "ChargeDetailID0_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1045", "mean": "332.25", "stddev": "253.21", "min": "0.0", "max": "815.0", "missing": "0"}, "updatedLabel": "ChargeDetailID0_stringind..."}, {"feature": "RVUModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1045", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "RVUModifier_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1045", "mean": "0.37", "stddev": "0.48", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "PayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1045", "mean": "3.59", "stddev": "7.03", "min": "0.0", "max": "47.0", "missing": "0"}, "updatedLabel": "PayerName_stringindexer_t..."}, {"feature": "PayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1045", "mean": "1.9", "stddev": "2.96", "min": "0.0", "max": "16.0", "missing": "0"}, "updatedLabel": "PayerCategory_stringindex..."}, {"feature": "PayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1045", "mean": "0.84", "stddev": "1.62", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "PayerGroup_stringindexer_..."}, {"feature": "SourceKey58_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1045", "mean": "6.46", "stddev": "11.16", "min": "0.0", "max": "69.0", "missing": "0"}, "updatedLabel": "SourceKey58_stringindexer..."}, {"feature": "ReasonDate_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1045", "mean": "10.45", "stddev": "12.99", "min": "0.0", "max": "73.0", "missing": "0"}, "updatedLabel": "ReasonDate_stringindexer_..."}, {"feature": "FacilityID63_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1045", "mean": "16.65", "stddev": "17.77", "min": "0.0", "max": "80.0", "missing": "0"}, "updatedLabel": "FacilityID63_stringindexe..."}, {"feature": "DepartmentID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1045", "mean": "1.87", "stddev": "3.05", "min": "0.0", "max": "23.0", "missing": "0"}, "updatedLabel": "DepartmentID_stringindexe..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run denialautoHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["ChargeDetailID0_stringindexer", "Amount1", "Amount2", "CPTCodeID3", "DiagnosisCodeID2", "DiagnosisCodeID1", "RVUModifier_stringindexer", "Gender_stringindexer", "EnterpriseID18", "PayerName_stringindexer", "PayerCategory_stringindexer", "PayerGroup_stringindexer", "SpecialityID", "TaxonomyID", "SourceKey58_stringindexer", "Amount59", "ReasonDate_stringindexer", "PracticeID", "FacilityID63_stringindexer", "CPTCodeID66", "DepartmentID_stringindexer"], "status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

